In [42]:
# Install the langgraph package if not already installed
%pip install langgraph graphviz notebook ipython pyppeteer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 11.4 MB/s eta 0:00:00
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 33.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.4.0
    Uninstalling urllib3-2.4.0:
      Successfully uninstalled urllib3-2.4.0

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [53]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [ ]:
from typing import TypedDict, List, Optional

class AgentState(TypedDict):
    query: str
    enhanced_query: Optional[str]
    vector_result: Optional[List[str]]
    graph_result: Optional[List[str]]
    validated: Optional[bool]
    final_output: Optional[str]
    messages: Annotated[list, add_messages]


In [31]:
# Mock agent functions
def run_enhancer_agent(state): return {**state, "query": f"Enhanced: {state['query']}"}
def run_supervisor_agent(state): return state  # add routing logic later
def run_vector_agent(state): return {**state, "vector_results": ["Movie A", "Movie B"]}
def run_graph_agent(state): return {**state, "graph_results": ["Movie C", "Movie D"]}
def run_evaluator_agent(state): return {**state, "final_results": list(set(state.get("vector_results", []) + state.get("graph_results", [])))}


In [ ]:

# Import necessary modules from langgraph
from langgraph.graph import StateGraph, END

def build_graph():
    builder = StateGraph(AgentState)
    
    builder.add_node("enhancer", run_enhancer_agent)
    builder.add_node("supervisor", run_supervisor_agent)
    builder.add_node("vector", run_vector_agent)
    builder.add_node("graph", run_graph_agent)
    builder.add_node("evaluator", run_evaluator_agent)

    builder.add_edge(START, "enhancer")
    builder.add_edge("enhancer", "supervisor")
    builder.add_edge("supervisor", "vector")
    builder.add_edge("supervisor", "graph")
    builder.add_edge("supervisor", "evaluator")
    builder.add_edge("supervisor", "evaluator")
    builder.add_edge("evaluator", END)

    return builder.compile()

graph = build_graph()

display(Image(graph.get_graph(xray=True).draw_mermaid_png())) 

In [ ]:
from graphviz import Source

# Visualize the graph 
def draw_langgraph():
    compiled = build_graph().compile()
    dot = compiled.get_graph().draw()
    return Source(dot)

from IPython.display import display
display(draw_langgraph())

In [41]:
from IPython.display import display,Image
display(Image(graph.get_graph(xray=True).draw_mermaid_png())) # Display the graph's PNG representation


ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`